# Preprocesamiento de datos

## Objetivos particulares:

1. Entender los datos que representan las columnas.
2. Implementar una política de datos faltantes.
3. Creación de columnas para crear mejor información.
4. Segmentar en regiones geográficas de México.
5. Eliminación de columnas con datos innecesarios.
6. Homogeneizar los datos.

## Librerías

In [58]:
import pandas as pd
import numpy as np
import random
import copy
from sklearn.preprocessing import LabelEncoder

## 1. Cargar el conjunto de datos ENDIREH Sección IX: Atención Obstétrica

In [59]:
ruta_parcial = "../conjunto_de_datos_endireh_2016_csv/conjunto_de_datos_tb_sec_ix_endireh_2016/conjunto_de_datos/conjunto_de_datos_tb_sec_ix_endireh_2016.csv"

endireh = pd.read_csv(ruta_parcial, na_values=['\r'])

In [60]:
endireh.iloc[random.choices(range(endireh.shape[0]), k=5)]

,ID_VIV,ID_MUJ,UPM,REN_M_ELE,VIV_SEL,PROG,HOGAR,DOMINIO,CVE_ENT,NOM_ENT,...,P9_8_10,P9_8_11,P9_8_12,P9_8_13,P9_8_14,FAC_VIV,FAC_MUJ,ESTRATO,UPM_DIS,EST_DIS
57788,3200225.04,3200225.04.01.05\r,3200225,5,4,80,1,U\r,32,Zacatecas\r,...,2.0,2.0,NaN,NaN,NaN,88,88,3,17650,6130
69419,1800679.04,1800679.04.01.02\r,1800679,2,4,109,1,U\r,18,Nayarit\r,...,NaN,NaN,NaN,NaN,NaN,92,92,4,9973,3230
3711,202831.03,0202831.03.01.02\r,202831,2,3,82,1,U\r,2,Baja California\r,...,2.0,1.0,1.0,1.0,NaN,268,268,2,916,200
67818,1702671.09,1702671.09.01.02\r,1702671,2,9,127,1,C\r,17,Morelos\r,...,2.0,2.0,NaN,NaN,NaN,138,138,2,9653,3180
103834,2760604.12,2760604.12.01.04\r,2760604,4,12,186,1,R\r,27,Tabasco\r,...,NaN,NaN,NaN,NaN,NaN,183,367,1,15268,5020


## 2. Preservar los datos de las embarazadas

Borrar los registros de todas aquellas que repondieron que no estuvieron embarazadas entre Octubre 2011 hasta finales 2016 o que no se tenga resgistro de su respuesta. La manera fácil es solo preservar los registros de las que respondieron que sí estuvieron embaraadas en ese periodo de tiempo.

In [61]:
endireh = endireh[endireh.P9_2 == 1]

In [62]:
endireh.reset_index(drop=True, inplace=True) #reajustar el índice

## 3. Agregar columnas

### 3.1 Datos faltantes a preguntas de servicio médico.

En la pregunta 1 y 5, si todos los registros tienen valor 9 significa que no hay registro de la respuesta a esa pregunta, por ello se grega una columna que refleje esa información y todos los demás valores se hacen 0, así se preserva el _One Hot Encoding_.

#### Pregunta 1

In [63]:
endireh.insert(24, "P9_1_10", 0.0) #agregar columna con ceros

In [64]:
endireh.loc[endireh['P9_1_1']==9, 'P9_1_10'] = 1.0 #asignar los valores de columnas con 9

In [65]:
endireh.loc[endireh['P9_1_1']==9, [F'P9_1_{i}' for i in range(1,10)]] = 0.0 #los valores con 9 se hacen 0

#### Pregunta 5

In [66]:
endireh.insert(41, "P9_5_12", 0.0) #agregar columna con ceros

In [67]:
endireh.loc[endireh['P9_5_1']==9, 'P9_5_12']=1.0 #asignar los valores de columnas con 9

In [68]:
endireh.loc[endireh['P9_5_1']==9, [F'P9_5_{i}' for i in range(1,12)]]=0.0 #los valores con 9 se hacen 0

### 3.2 Dividir por regiones de méxico.

Para no dividir entre 32 entidades federativas, dividiré el país en principales regiones de México.
Según la página de [Regiones de México](https://es.wikipedia.org/wiki/Regiones_de_M%C3%A9xico) de Wikipedia existen un total de 8 regiones.
   

|  Noroeste de México | Noreste de México  | Oeste de México  | Este de México  |
|:-------------------:|:------------------:|:----------------:|:---------------:|
| Baja California     | Coahuila           | Colima           | Hidalgo         |
| Baja California Sur | Nuevo León         | Jalisco          | Puebla          |
| Chihuahua           | Tamaulipas         | Michoacán        | Tlaxcala        |
| Durango             |                    | Nayarit          | Veracruz        |
| Sinaloa             |                    |                  |                 |
| Sonora              |                    |                  |                 |

|  Centronorte de México  | Centrosur de México  | Suroeste de México  | Sureste de México  |
|:-----:                  |:--------------------:|:-------------------:|:------------------:|
| Aguascalientes          | Ciudad de México     | Chiapas             | Campeche           |
| Guanajuato              | México               | Guerrero            | Quintana Roo       |
| Querétaro               | Morelos              | Oaxaca              | Tabasco            |
| San Luis Potosí         |                      |                     | Yucatán            |
| Zacatecas               |                      |                     |                    |

Haré un total de 8 archivos _csv_ en los que estarán en cada uno los datos de los estados correspondientes a la región del país.

Creo un código para rastrear el número de región a su significado.


|  Código  | Región         |
|:--------:|:--------------:|
| 0        | Noroeste       |
| 1        | Noreste        |
| 2        | Oeste          |
| 3        | Este           |
| 4        | Centronorte    |
| 5        | Centrosur      |
| 6        | Suroeste       |
| 7        | Sureste        |


Crear la nueva columna de región

In [69]:
def f(row):
    if   row['NOM_ENT'] in ['Baja California\r', 'Baja California Sur\r', 'Chihuahua\r', 'Durango\r', 'Sinaloa\r', 'Sonora\r']:
        val = 0
    elif row['NOM_ENT'] in ['Coahuila de Zaragoza\r', 'Nuevo Leon\r', 'Tamaulipas\r']:
        val = 1
    elif row['NOM_ENT'] in ['Colima\r', 'Jalisco\r', 'Michoacan de Ocampo\r', 'Nayarit\r']:
        val = 2
    elif row['NOM_ENT'] in ['Hidalgo\r', 'Puebla\r', 'Tlaxcala\r', 'Veracruz de Ignacio de la Llave\r']:
        val = 3
    elif row['NOM_ENT'] in ['Aguascalientes\r', 'Guanajuato\r', 'Queretaro\r', 'San Luis Potosi\r', 'Zacatecas\r']:
        val = 4
    elif row['NOM_ENT'] in ['Ciudad de Mexico\r', 'Estado de Mexico\r', 'Morelos\r']:
        val = 5
    elif row['NOM_ENT'] in ['Chiapas\r', 'Guerrero\r', 'Oaxaca\r']:
        val = 6
    elif row['NOM_ENT'] in ['Campeche\r', 'Quintana Roo\r', 'Tabasco\r', 'Yucatan\r']:
        val = 7
        
    return val

In [70]:
endireh_reg = copy.copy(endireh)

In [71]:
endireh_reg.insert(12, "REGION", endireh.apply(f, axis=1))

In [72]:
endireh_reg.head(10)

,ID_VIV,ID_MUJ,UPM,REN_M_ELE,VIV_SEL,PROG,HOGAR,DOMINIO,CVE_ENT,NOM_ENT,...,P9_8_10,P9_8_11,P9_8_12,P9_8_13,P9_8_14,FAC_VIV,FAC_MUJ,ESTRATO,UPM_DIS,EST_DIS
0,100097.01,0100097.01.01.02\r,100097,2,1,18,1,U\r,1,Aguascalientes\r,...,2.0,1.0,1.0,1.0,NaN,82,82,2,28,10
1,100101.04,0100101.04.01.03\r,100101,3,4,51,1,U\r,1,Aguascalientes\r,...,2.0,1.0,1.0,2.0,4.0,94,189,3,29,20
2,100105.03,0100105.03.01.02\r,100105,2,3,171,1,U\r,1,Aguascalientes\r,...,1.0,2.0,NaN,NaN,NaN,85,85,3,31,20
3,100109.01,0100109.01.01.04\r,100109,4,1,21,1,U\r,1,Aguascalientes\r,...,2.0,1.0,1.0,1.0,NaN,92,185,3,32,20
4,100109.03,0100109.03.01.02\r,100109,2,3,33,1,U\r,1,Aguascalientes\r,...,2.0,1.0,1.0,1.0,NaN,92,92,3,32,20
5,100003.03,0100003.03.01.02\r,100003,2,3,61,1,U\r,1,Aguascalientes\r,...,2.0,1.0,1.0,1.0,NaN,115,115,4,2,30
6,100018.04,0100018.04.01.02\r,100018,2,4,115,1,U\r,1,Aguascalientes\r,...,2.0,2.0,NaN,NaN,NaN,92,92,3,7,20
7,100025.01,0100025.01.01.07\r,100025,7,1,9,1,U\r,1,Aguascalientes\r,...,2.0,2.0,NaN,NaN,NaN,92,92,3,9,20
8,100025.02,0100025.02.01.02\r,100025,2,2,28,1,U\r,1,Aguascalientes\r,...,2.0,1.0,1.0,1.0,NaN,92,92,3,9,20
9,100025.05,0100025.05.01.03\r,100025,3,5,68,1,U\r,1,Aguascalientes\r,...,2.0,1.0,1.0,1.0,NaN,92,184,3,9,20


### 3.3 Crear clasificación

A partir de las respuestas de la pregunta 8, crear columna que va a servir de clasificación para los niveles de violencia obstétrica.

In [73]:
def f(row):
    # va de menor a mayor gravedad
    val = 0 #ninguno
    if row['P9_8_1'] == 1.0 or row['P9_8_2'] == 1.0 or row['P9_8_3'] == 1.0 or row['P9_8_6'] == 1.0: #humillante
        val = 1
    if row['P9_8_7'] == 1.0 or row['P9_8_8'] == 1.0: #salud reproductiva
        val = 2
    if row['P9_8_4'] == 1.0 or row['P9_8_5'] == 1.0 or row['P9_8_10'] == 1.0: #atencion medica
        val = 3
    if row['P9_8_9'] == 1.0 or row['P9_8_14'] == 4.0: #grave
        val = 4
        
    return val

In [74]:
endireh.insert(58, "P9_8", endireh.apply(f, axis=1))

Borrar las preguntas ya tomadas en cuenta en lo anterior. Preguntas de la 1-10 y 12.
Las preguntas, 11, 13 y 14 pueden ser parte de las variables.

In [75]:
endireh.drop(labels=[f'P9_8_{i}' for i in range(1,11)], axis=1, inplace=True)

In [76]:
endireh.drop(labels='P9_8_12', axis=1, inplace=True)

## 4. Borrar columnas

### 4.1  Las columnas con datos que ya son representados en otras columnas son:

<center>[NOM_ENT, NOM_MUN]</center>

Eliminar las columnas de nombre de municipio y nombre del estado que ya está representados con números en otras columnas.


In [77]:
endireh_reg.drop(labels=['NOM_ENT', 'NOM_MUN'], axis=1, inplace=True)

### 4.2 Las columnas con datos innecesarios:

<center>[COD_RES_MU, COD_RES, P9_2]</center>

Eliminar las columnas cuyos registros solo tomen un valor. En este caso las columnas que representan el estatus de haber sido completadas y de estar embarazada (cuya variabilidad de acorto en el inciso 2).

In [78]:
endireh_reg.drop(labels=['COD_RES_MU', 'COD_RES', 'P9_2'], axis=1, inplace=True)

### 4.3 Las columnas de violencia obtétrica durante el parto:

<center>[P9_8_1, ..., P9_8_10, P9_8_12]</center>

Eliminar las columnas cuyos registros se utilisen para la creación de la clasificación. Preservando los datos de si el último hijo nació por cesárea, y quién dió la autorización para la cesárea.

In [81]:
columnas = [f'P9_8_{i}' for i in range(1,11)]
columnas.append('P9_8_12')

In [82]:
endireh_reg.drop(labels=columnas, axis=1, inplace=True)

## 5. Limpiar columnas

Las columnas con datos por limpiar (que sus datos terminan en __\r__) :

Usamos expresiones regulares para quedarnos con la parte de la expresión que no tiene el termino \r

In [83]:
endireh_reg_r = copy.copy(endireh_reg)

In [84]:
endireh_reg_r = endireh_reg.replace(regex=r'(\w+)(\W)*\r', value=r'\1')

In [85]:
endireh_reg_r.head(10)

,ID_VIV,ID_MUJ,UPM,REN_M_ELE,VIV_SEL,PROG,HOGAR,DOMINIO,CVE_ENT,CVE_MUN,...,P9_6,P9_7,P9_8_11,P9_8_13,P9_8_14,FAC_VIV,FAC_MUJ,ESTRATO,UPM_DIS,EST_DIS
0,100097.01,0100097.01.01.02,100097,2,1,18,1,U,1,1,...,2016.0,5.0,1.0,1.0,NaN,82,82,2,28,10
1,100101.04,0100101.04.01.03,100101,3,4,51,1,U,1,1,...,2012.0,5.0,1.0,2.0,4.0,94,189,3,29,20
2,100105.03,0100105.03.01.02,100105,2,3,171,1,U,1,1,...,2015.0,2.0,2.0,NaN,NaN,85,85,3,31,20
3,100109.01,0100109.01.01.04,100109,4,1,21,1,U,1,1,...,2012.0,5.0,1.0,1.0,NaN,92,185,3,32,20
4,100109.03,0100109.03.01.02,100109,2,3,33,1,U,1,1,...,2013.0,6.0,1.0,1.0,NaN,92,92,3,32,20
5,100003.03,0100003.03.01.02,100003,2,3,61,1,U,1,1,...,2016.0,6.0,1.0,1.0,NaN,115,115,4,2,30
6,100018.04,0100018.04.01.02,100018,2,4,115,1,U,1,1,...,2014.0,5.0,2.0,NaN,NaN,92,92,3,7,20
7,100025.01,0100025.01.01.07,100025,7,1,9,1,U,1,1,...,2011.0,5.0,2.0,NaN,NaN,92,92,3,9,20
8,100025.02,0100025.02.01.02,100025,2,2,28,1,U,1,1,...,2013.0,6.0,1.0,1.0,NaN,92,92,3,9,20
9,100025.05,0100025.05.01.03,100025,3,5,68,1,U,1,1,...,2014.0,2.0,1.0,1.0,NaN,92,184,3,9,20


## 6. Creo los datasets de cada region.

In [87]:
gk = endireh_reg_r.groupby(by='REGION', axis=0)

### 6.1 Guardar los datasets

In [88]:
#guardo los avances
endireh_reg_r.to_csv("../datasets/sec_ix.csv")

In [89]:
for grupo in endireh_reg_r['REGION'].unique():
    gk.get_group(grupo).to_csv(f"datasets/sec_ix_region_{grupo}.csv")